In [1]:
!pip install accelerate bitsandbytes sentence_transformers faiss-cpu langchain langchain_community --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 77.4 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')
# Đọc một tệp tin từ Google Drive
path = '/content/drive/MyDrive/Colab_Notebooks'

Mounted at /content/drive


In [ ]:
# import zipfile
# import os

# # Define the path to the zip file and the extraction directory
# zip_file_path = path + '/10k_doc_wiki_tieng_viet.zip'
# extraction_path = path + '/10k_doc2'

# # Create the extraction directory if it does not exist
# if not os.path.exists(extraction_path):
#     os.makedirs(extraction_path)

# # Extract the zip file
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extraction_path)

# print(f"Files extracted to: {extraction_path}")


Files extracted to: /content/drive/MyDrive/Colab_Notebooks/10k_doc2


In [ ]:
from langchain.embeddings.base import Embeddings
from sentence_transformers import SentenceTransformer
from typing import List

class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name: str):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, documents: List[str]) -> List[List[float]]:
        return self.model.encode(documents)

    def embed_query(self, query: str) -> List[float]:
        return self.model.encode([query])[0]

    def encode(self, text):
        return self.model.encode(text, normalize_embeddings=True)

embedding_model = SentenceTransformerEmbeddings("BAAI/bge-m3")

In [7]:
sentences_1 = ["What is BGE M3?", "Defination of BM25"]
sentences_2 = ["BGE M3 is an embedding model supporting dense retrieval, lexical matching and multi-vector interaction.",
               "BM25 is a bag-of-words retrieval function that ranks a set of documents based on the query terms appearing in each document"]

embeddings_1 = embedding_model.encode(sentences_1,
                            # normalize_embeddings=True
                           )
embeddings_2 = embedding_model.encode(sentences_2,
                            # normalize_embeddings=True
                            )
similarity = embeddings_1 @ embeddings_2.T
print(similarity)
# [[0.6265, 0.3477], [0.3499, 0.678 ]]


[[0.6259037  0.34749576]
 [0.34986788 0.6782464 ]]


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import GPT4AllEmbeddings

# Khai bao bien
data_path = path + "/10k_doc2/data_raw10k"
vector_db_path = path + "/vectorstores/db_faiss"

def create_db_from_files():
    # Check if the directory exists
    if not os.path.exists(data_path):
        raise FileNotFoundError(f"The directory {data_path} does not exist.")

    # Initialize the directory loader to scan the data directory
    loader = DirectoryLoader(data_path, glob="*.txt", loader_cls = TextLoader)

    # Load documents
    documents = loader.load()
    if not documents:
        raise ValueError("No documents found in the specified directory.")

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
    chunks = text_splitter.split_documents(documents)

    db = FAISS.from_documents(chunks, embedding = embedding_model)
    db.save_local(vector_db_path)

create_db_from_files()


In [10]:
from langchain_community.vectorstores import FAISS

# Load the vector database
vector_db_path = path + "/vectorstores/db_faiss"
db = FAISS.load_local(vector_db_path, embeddings=embedding_model, allow_dangerous_deserialization=True)

def search_top_k(query, k=3):
    # Generate query vector
    query_embedding = embedding_model.embed_query(query)

    # Search for top k candidates
    results = db.similarity_search_by_vector(query_embedding, k)

    return results

# Example query
query = "Ai là tác giả của 18 món quà"
top_k_results = search_top_k(query, k=5)

for result in top_k_results:
    print(result)

page_content='18 món quà' metadata={'source': '/content/drive/MyDrive/Colab_Notebooks/10k_doc2/data_raw10k/18 món quà.txt'}
page_content='18 món quà ( ) là một bộ phim chính kịch Ý năm 2020 do Francesco Amato viết kịch bản và đạo diễn. Vittoria Puccini, Benedetta Porcaroli và Edoardo Leo tham gia diễn xuất các vai chính. Bộ phim dựa trên câu chuyện có thật của một người phụ nữ Ý mang tên Elisa Girotto. Elisa đã lên kế hoạch mua và chia đều quà sinh nhật 17 năm mỗi năm một lần cho con gái Anna trước khi cô qua đời vào tháng 9 năm 2017 vì căn bệnh ung thư vú giai đoạn cuối. Girotto được chẩn đoán mắc bệnh ung thư vào thời điểm cô sinh con gái vào tháng 8 năm 2016 ở tuổi 40. Bộ phim được phát hành vào ngày 2 tháng 1 năm 2020 và nhận được những đánh giá tích cực về kịch bản phim đầy cảm xúc. Chồng của Elisa, Alessio Vincenzotto cũng ghi công trong phim nhờ câu chuyện về vợ mình.Bộ phim còn được phát trực tuyến trên Netflix vào ngày 8 tháng 5 năm 2020, là Ngày của mẹ ở Hoa Kỳ.\nNội dung.'

In [11]:
import os
from google.colab import userdata

secret_value_0 = userdata.get('HUGGINGFACE_TOKEN')

# Set the environment variable for the Hugging Face token
# os.environ['HUGGINGFACE_TOKEN'] = secret_value_0
from huggingface_hub import login
login(token=secret_value_0)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
# lora_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

lora_config = BitsAndBytesConfig(
    load_in_4bit=True,
    # load_in_8bit=True,
)

# model_name = 'google/gemma-1.1-2b-it'
model_name = 'Viet-Mistral/Vistral-7B-Chat'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=lora_config,
#     torch_dtype=torch.bfloat16, # change to torch.float16 if you're using V100
    device_map="auto",
    use_cache=True,
)

In [26]:
import warnings
warnings.filterwarnings('ignore')

system_prompt = "Bạn là một trợ lí Tiếng Việt nhiệt tình và trung thực. Hãy luôn trả lời một cách hữu ích nhất có thể, đồng thời giữ an toàn.\n"
system_prompt += "Câu trả lời của bạn không nên chứa bất kỳ nội dung gây hại, phân biệt chủng tộc, phân biệt giới tính, độc hại, nguy hiểm hoặc bất hợp pháp nào. Hãy đảm bảo rằng các câu trả lời của bạn không có thiên kiến xã hội và mang tính tích cực."
system_prompt += "Nếu một câu hỏi không có ý nghĩa hoặc không hợp lý về mặt thông tin, hãy giải thích tại sao thay vì trả lời một điều gì đó không chính xác. Nếu bạn không biết câu trả lời cho một câu hỏi, hãy trả lời là bạn không biết và vui lòng không chia sẻ thông tin sai lệch."

system_prompt = "Tôi là một trợ lí Tiếng Việt nhiệt tình và trung thực. Tôi luôn trả lời một cách hữu ích nhất có thể, đồng thời giữ an toàn.\n"
# system_prompt += "Câu trả lời của tôi không chứa bất kỳ nội dung gây hại, phân biệt chủng tộc, phân biệt giới tính, độc hại, nguy hiểm hoặc bất hợp pháp nào. Tôi luôn đảm bảo rằng các câu trả lời của tôi không có thiên kiến xã hội và mang tính tích cực."
# system_prompt += "Nếu một câu hỏi không có ý nghĩa hoặc không hợp lý về mặt thông tin, tôi sẽ giải thích tại sao thay vì trả lời một điều gì đó không chính xác. Nếu tôi không biết câu trả lời cho một câu hỏi, tôi sẽ trả lời là tôi không biết và không chia sẻ thông tin sai lệch."

conversation = [{"role": "system", "content": system_prompt }]

print("Xin chào, tôi là trợ lý chatbot của bạn. Tôi có thể giúp gì cho bạn không?")

conversation = []

while True:
    human = input("You: ")
    if human.lower() == "reset":
        # conversation = [{"role": "system", "content": system_prompt }]
        print("The chat history has been cleared!")
        continue
    elif human.lower() == "quit":
        break

    conversation.append({"role": "user", "content": human })
    input_ids = tokenizer.apply_chat_template(conversation, return_tensors="pt").to(model.device)

    # Find appropreate context
    context = ""
    if human[-1] == '?':
        context = '\n'.join(x.page_content for x in search_top_k(human, 3))
        print(context)
        # print("context: ", context)
        # Add context to conversation
        # conversation.append({"role": "assistant", "content": f"Dưới đây là một vài văn bản có thể chứa thông tin liên quan đến câu hỏi của bạn: {context}" })
        # conversation.append({"role": "user", "content": f'Vậy dựa vào đoạn văn bản trên mà bạn đã tìm được, hãy cho tôi biết {human}' })

        conversation = [{"role": "system", "content": f'Sử dụng thông tin sau đây để trả lời câu hỏi. Nếu bạn không biết câu trả lời, hãy nói không biết, đừng cố tạo ra câu trả lời\n {context}'}]
        conversation.append({"role": "user", "content": human })

    out_ids = model.generate(
        input_ids=input_ids,
        max_new_tokens=768,
        do_sample=True,
        top_p=0.9,
        top_k=10,
        temperature=0.1,
        repetition_penalty=1.05,
        pad_token_id=tokenizer.eos_token_id,
    )
    assistant = tokenizer.batch_decode(out_ids[:, input_ids.size(1): ], skip_special_tokens=True)[0].strip()
    print("Assistant: ", assistant)
    conversation.append({"role": "assistant", "content": assistant })

print("Hẹn gặp lại bạn vào lần sau!")

Xin chào, tôi là trợ lý chatbot của bạn. Tôi có thể giúp gì cho bạn không?
You: Giám đốc hiện tại của Đại học Bách khoa Hà Nội là ai?
Ngày 22/4/2019, PGS.TS Nguyễn Thị Kim Tiến trao Quyết định số 1368/QĐ-BYT ngày 12/4/2019 của Bộ trưởng Bộ Y tế bổ nhiệm chức danh Giám đốc Bệnh viện Đại học Y Hà Nội nhiệm kỳ 2019 - 2024 cho PGS.TS Nguyễn Lân Hiếu.
Đại biểu Quốc hội Việt Nam khóa 14 nhiệm kì 2016-2021.
Ông được Tổng hội Y học Việt Nam để cử tham gia ứng cử ĐBQH khóa 14. Sau khi vượt qua 3 vòng hiệp thương, ông được giới thiệu về tỉnh An Giang ứng cử ở đơn vị bầu cử số 2 tỉnh An Giang gồm các huyện: Châu Phú và Châu Thành. Tại cuộc bầu cử ngày 22.5.2016, ông Hiếu đạt 63,62% số phiếu bầu và trúng cử.
Ngày 12 tháng 6 năm 2017, ông kiến nghị phải có một kì thi chung quốc gia cho sinh viên ngành Y để kiểm soát chất lượng đầu ra của ngành này.
Đề nghị lùi thông qua Luật An ninh mạng.
Từ năm 2013 đến năm 2016, ông tiếp tục đảm nhận vị trí giám đốc trung tâm nội soi tại Bệnh viện Đại học Y Hà Nộ

In [ ]:
Nam bao nhiều tuổi thì được đăng ký kết hôn?
Giám đốc hiện tại của Đại học Bách khoa Hà Nội là ai?
Đạo diễn của 18 món quà là ai